In [ ]:
# This notebook will allow you to enter a city of your choice from anywhere in the world and receive the current weather details for that location.

In [ ]:
!pip install --user requests


In [ ]:
# -*- coding: utf-8 -*-
api_key = '00e9c642575dfb0616fef82397a57a47'
#tz = 'Europe/London'

import requests
#import json

weather_components = ["weather", "main", "visibility", "wind","rain"]
exclude_list = ["icon", "id"  ]
separator = "============================"

def recurrent_print_datatype(component , weather_descriptor):
        
    if isinstance(weather_descriptor, str) or isinstance(weather_descriptor, int):
        print(f"{component} - {weather_descriptor}")
        
    elif isinstance(weather_descriptor, list):
        for item in weather_descriptor:            
            if isinstance(item, dict):
                for key, value in item.items():
                    if key not in exclude_list:
                        print(f"{key} - {value}")
            else:
                for value in weather_descriptor:
                    if value not in exclude_list:
                        print(f"{value}")
                        
    elif isinstance(weather_descriptor, dict):
        for key, value in weather_descriptor.items():
            if key not in exclude_list:
                print(f"{key} - {value}")        
    


def is_city(city_name):
    print("You entered:", city_name)        
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name.lower()}&appid={api_key}&units=metric"
    response = requests.get(url)    
    data = response.json()    
    if data["cod"] != 200:
        print("Error: ", data["message"])
        return False 
          
    for component in weather_components:
        if component in data.keys():            
            weather_descriptor = data[component]            
            recurrent_print_datatype(component , weather_descriptor)
    print(separator)            
    return data["cod"] == 200

#is_city("London")
   

In [ ]:
while True:
    city = input("Enter a city name (or 'exit' to quit): ")
    if city.lower() == "exit" or city.lower() == "quit":
        break
    else:        
        if len(city) == 0:            
            print("You must enter a city name.")
            continue
        elif not is_city(city):
            print("Information about the city was not found. Please try again.")
            continue    
        